In [ ]:
!pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 10.2 MB/s eta 0:00:00


In [ ]:
!pip install qiskit_aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 35.3 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer

In [ ]:
from qiskit import transpile

In [ ]:
import numpy as np

In [ ]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.visualization import plot_histogram
import math
import random

# Define the integer to be factored
n = 35

# Determine the number of qubits needed
num_qubits = math.ceil(math.log2(n))

# Define the quantum circuit
qc = QuantumCircuit(num_qubits * 2 + 1, num_qubits)

# Apply the Hadamard gates to the first N qubits
for i in range(num_qubits):
    qc.h(i)

# Apply the controlled-U gates
a = 2
qc.cp(math.pi / n, num_qubits, num_qubits + 1)
qc.cp(math.pi / n, num_qubits, num_qubits + 1)
qc.cp(-math.pi / n, num_qubits, num_qubits + 1)
qc.cp(math.pi / n, num_qubits, num_qubits + 1)

# Apply the inverse quantum Fourier transform
qc.swap(num_qubits, num_qubits + 1)
qc.h(num_qubits + 1)
for i in range(num_qubits // 2):
    qc.cp(-math.pi / 2, i, num_qubits + 1)
    qc.cp(-math.pi / 4, i, num_qubits + 1)
    qc.cp(-math.pi / 2, i, num_qubits + 1)
    qc.cp(-math.pi / 4, i, num_qubits + 1)
for i in range(num_qubits // 2, num_qubits):
    qc.cp(-math.pi / 4, i, num_qubits + 1)
    qc.cp(-math.pi / 2, i, num_qubits + 1)
    qc.cp(-math.pi / 4, i, num_qubits + 1)
    qc.cp(-math.pi / 2, i, num_qubits + 1)

# Measure the first N qubits
for i in range(num_qubits):
    qc.measure(i, i)

# Transpile the quantum circuit
qc_transpiled = transpile(qc, Aer.get_backend('qasm_simulator'))

# Assemble the transpiled circuit for execution
qobj = assemble(qc_transpiled)

# Execute the transpiled circuit on a simulator
simulator = Aer.get_backend('qasm_simulator')
result = simulator.run(qobj).result()

# Get the counts from the result
counts = result.get_counts()

# Find the most likely state
most_likely_state = max(counts, key=counts.get)

# Calculate the period using the most likely state
power_of_a = int(most_likely_state, 2)
period = 0
for i in range(1, num_qubits + 1):
    if (power_of_a & (1 << (num_qubits + 1 - i))) != 0:
        period = 2 ** i
        break

# Calculate the non-trivial factors
u = (a ** (period // 2) - 1) % n
v = (a ** (period // 2) + 1) % n
gcd_u = math.gcd(u, n)
gcd_v = math.gcd(v, n)

# Print the non-trivial factors
if gcd_u > 1:
    print("The non-trivial factors of", n, "are", gcd_u, "and", n // gcd_u)
elif gcd_v > 1:
    print("The non-trivial factors of", n, "are", gcd_v, "and", n // gcd_v)
else:
    print("No non-trivial factors found")

The non-trivial factors of 35 are 5 and 7


<ipython-input-8-11e7a30dcc1b>:52: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.14 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = simulator.run(qobj).result()


Importing Required Libraries: The code imports necessary modules from the Qiskit library for quantum computing, including QuantumCircuit for creating quantum circuits, transpile for optimizing circuits, assemble for assembling circuits for execution, Aer for accessing Qiskit's simulation backends, and plot_histogram for visualizing measurement results.
Defining the Integer to be Factored: The integer n to be factored is set to 35. Shor's algorithm is designed to efficiently factorize large integers into their prime factors, but this example focuses on a small integer for simplicity.
Determining the Number of Qubits: The number of qubits needed for the quantum circuit is determined based on the binary representation of the integer n.
Creating the Quantum Circuit: The quantum circuit (qc) is created with num_qubits * 2 + 1 qubits. Hadamard gates (H) are applied to the first num_qubits qubits to create a superposition of all possible states. Controlled-U gates are applied to implement modular exponentiation.
Applying Controlled-U Gates: Controlled-U gates are used to perform modular exponentiation, where a is the base and n is the modulus. The controlled-U gates implement the function f(x) = a^x mod n.
Applying Inverse Quantum Fourier Transform (QFT): The inverse quantum Fourier transform is applied to the second set of num_qubits qubits. This step involves applying a series of controlled-phase gates (CU1) and Hadamard gates (H).
Measuring the Qubits: The first num_qubits qubits are measured to obtain the measurement outcomes.
Transpiling the Circuit: The quantum circuit is transpiled to optimize it for a specific backend.
Assembling the Transpiled Circuit: The transpiled circuit is assembled into a qobj for execution.
Executing the Circuit on a Simulator: The transpiled circuit is executed on a quantum simulator backend (qasm_simulator) provided by Aer.
Analyzing the Measurement Results: The measurement results are obtained, and the most likely state (the one with the highest probability) is determined.
Calculating the Period: The most likely state is used to calculate the period of the modular exponentiation function.
Calculating the Non-Trivial Factors: Using the period, the non-trivial factors of n are calculated using simple arithmetic.
Printing the Results: The non-trivial factors are printed as the output.

In [ ]:
import math

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def pollard_rho(N):
    if N % 2 == 0:
        return 2, N // 2

    x = random.randint(1, N-1)
    y = x
    c = random.randint(1, N-1)
    d = 1

    f = lambda x: (x ** 2 + c) % N

    while d == 1:
        x = f(x)
        y = f(f(y))
        d = gcd(abs(x-y), N)

    if d == N:
        return None

    return d, N // d

# Sample Input
N = 35

# Factorize using Pollard's rho algorithm
factor1, factor2 = pollard_rho(N)

# Print the output
print("Prime factors of", N, "are:", factor1, "and", factor2)


Prime factors of 35 are: 7 and 5


Import Necessary Libraries: We start by importing the math module, which provides mathematical functions like gcd, and the random module for generating random numbers.
Define the GCD Function: We define a function gcd to compute the greatest common divisor of two integers using the Euclidean algorithm. This function repeatedly applies the property gcd(a, b) = gcd(b, a % b) until b becomes zero, and then returns a.
Implement Pollard's Rho Algorithm: The main function pollard_rho takes an integer N as input and attempts to factorize it into its prime factors. If N is even, it immediately returns 2 and N // 2 as factors since 2 is the only even prime number.
Randomly Choose Starting Points: We start with a randomly chosen integer x between 1 and N-1, and initialize two variables y and d. We also select a random integer c between 1 and N-1.
Generate a Sequence of Numbers: We define a function f(x) which generates the next element in the sequence based on the current element x. In this implementation, f(x) = (x^2 + c) % N. We repeatedly apply this function to update x and y, where y updates twice as fast as x.
Find Greatest Common Divisor: At each iteration, we compute the greatest common divisor d of the absolute difference between x and y and N. If d is not 1 or N, then d is a non-trivial factor of N.
Repeat Until Non-Trivial Factor is Found: We continue the iterations until a non-trivial factor is found. If no such factor is found after a certain number of iterations, the algorithm returns None.
Return Prime Factors: If a non-trivial factor d is found, the function returns d and N // d, which are the prime factors of N.
Sample Input and Output: We provide a sample input N = 35 and call the pollard_rho function to factorize it. The prime factors factor1 and factor2 are then printed as output.

To compare the efficiency of both algorithms (Pollard's rho algorithm and Shor's algorithm), we need to consider several factors such as the size of the input number N, the computational resources available, and the specific implementations of each algorithm. Here's how we can compare them:

Number of Operations:
Pollard's rho algorithm typically requires fewer operations compared to Shor's algorithm for smaller input numbers. It has a time complexity of approximately O(sqrt(N)), where N is the input number.
Shor's algorithm, on the other hand, has a time complexity of approximately O((log N)^3), making it more efficient for large input numbers with a large number of digits.
Time Taken:
For relatively small input numbers (e.g., less than 100 digits), Pollard's rho algorithm can quickly factorize the number in a matter of milliseconds to seconds.
Shor's algorithm, while theoretically more efficient for large numbers, requires a quantum computer to execute. The time taken depends on the quantum computer's hardware, the implementation of the algorithm, and the size of the input number. Currently, quantum computers have limited qubits and decoherence times, making it impractical to factorize large numbers.
Computational Resources:
Pollard's rho algorithm can be executed on classical computers with standard hardware and software environments.
Shor's algorithm requires a quantum computer with a sufficient number of qubits and low error rates. As of now, quantum computers capable of factoring large numbers using Shor's algorithm are not yet available commercially, and existing quantum computers have limitations in terms of the number of qubits and coherence times.
In summary, for small input numbers, Pollard's rho algorithm is more practical and efficient as it can be executed on classical computers with minimal resources. However, for very large input numbers, Shor's algorithm theoretically offers superior efficiency but requires a functioning quantum computer, which is not widely available at present.

Advantages of Quantum Computing for Factoring Large Numbers:

Efficiency: Shor's algorithm offers exponential speedup over classical factoring algorithms for large numbers. Its time complexity scales polynomially with the number of digits in the input number, making it theoretically much faster than classical algorithms like Pollard's rho algorithm.
Parallelism: Quantum computers can perform computations on many possible inputs simultaneously through superposition and entanglement, allowing them to explore a vast solution space in parallel. This inherent parallelism is leveraged by Shor's algorithm to factorize large numbers efficiently.
Quantum Fourier Transform: Shor's algorithm exploits the quantum Fourier transform to find the periodicity of modular exponentiation efficiently. This enables the algorithm to identify the factors of a large composite number by leveraging the periodic nature of certain mathematical functions.
Limitations of Quantum Computing for Factoring Large Numbers:

Hardware Constraints: Current quantum computers have limited qubits and decoherence times, which restrict the size of the numbers they can factorize using Shor's algorithm. Factoring large numbers requires a significant number of qubits and precise quantum operations, which exceed the capabilities of existing quantum hardware.
Error Rates: Quantum computations are susceptible to errors due to factors such as noise, decoherence, and imperfect gates. As the size of the computation increases, so does the probability of errors accumulating and affecting the accuracy of the result. This poses a significant challenge for factoring large numbers using quantum algorithms.
Resource Intensiveness: Implementing Shor's algorithm requires a quantum computer with a large number of qubits and high-fidelity gates. Developing and maintaining such quantum hardware is resource-intensive and technically challenging. Additionally, the overhead involved in error correction and fault tolerance further complicates the practical implementation of Shor's algorithm.
Current State of Quantum Technology: Despite advances in quantum computing, practical quantum computers capable of factoring large numbers using Shor's algorithm are still in the early stages of development. Existing quantum hardware is not yet powerful enough to factorize numbers of cryptographic significance, limiting the immediate applicability of quantum factoring algorithms.
In conclusion, while quantum computing holds the potential for exponentially faster factoring of large numbers compared to classical computing, current technological limitations and challenges need to be addressed before quantum algorithms like Shor's algorithm can be effectively applied to practical cryptography and number theory problems. Continued research and development in quantum hardware, error correction, and algorithm optimization are essential for realizing the full potential of quantum computing in this domain.